# Ejercicio protocolo smtp con smtplib

Puedes encontrar toda la documentación de la librería `smtplib` en la web oficial de Python:
[https://docs.python.org/3/library/smtplib.html](https://docs.python.org/3/library/smtplib.html)

## Mi primer email con smtplib

1. Arranca un servidor smtp en local con el siguiente comando:
```
python -m smtpd -c DebuggingServer -n localhost:1025
```

2. Importa la librería smptlib 

In [1]:
import smtplib

3. Define las variables `smtp_server` y `port`.  
  - `smtp_server` debe contener un string con el nombre del servidor (mira el comando con el que has arrancado el servidor en el paso 1)
  - `port` debe contener un número entero con el puerto donde se ha arrancado el servidor (paso 1 también)

In [2]:
smtp_server  = 'localhost'
port = 1025

4. Define las siguientes variables:
  - `sender_email`: dirección del remitente (puedes inventartela) - tipo string
  - `receiver_email`: direccion del destinatario (puedes inventartela también) - tipo string
  - `message`: contenido del mensaje - 
    - puedes incluir el Subject en el cuerpo del mensaje
    - si quieres enviar un mensaje de varias líneas, prueba a definir el mensaje entre tres comillas consecutivas
    - por ejemplo:  
      ```
      message = """
          Subject: Hi there
          This message is sent from Python."""
      ```

In [3]:
sender_email = "my@gmail.com"
receiver_email = "your@gmail.com"
message = """
Subject: Hi there
This message is sent from Python."""

5. Envía el mensaje al servidor utilizando el siguiente código:

In [4]:
with smtplib.SMTP(smtp_server, port) as server:
    server.sendmail(sender_email, receiver_email, message)

En la consola donde habías arrancado el servidor, deberías ver algo como esto:
```
---------- MESSAGE FOLLOWS ----------                                                                                b'X-Peer: 127.0.0.1'                                                                                                 b''                                                                                                                  b'Subject: Hi there'                                                                                                 b'This message is sent from Python.'                                                                                 ------------ END MESSAGE ------------  
```

6. Para entender bien qué pasos ha seguido el protocolo "por dentro", añade la siguiente línea al código justo antes de mandar el mensaje:
```
server.set_debuglevel(True)
```

In [5]:
with smtplib.SMTP(smtp_server, port) as server:
    server.set_debuglevel(True)
    server.sendmail(sender_email, receiver_email, message)

send: 'ehlo DESKTOP-89DFD33.localdomain\r\n'
reply: b'250-DESKTOP-89DFD33.localdomain\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250 HELP\r\n'
reply: retcode (250); Msg: b'DESKTOP-89DFD33.localdomain\n8BITMIME\nHELP'
send: 'mail FROM:<my@gmail.com>\r\n'
reply: b'250 OK\r\n'
reply: retcode (250); Msg: b'OK'
send: 'rcpt TO:<your@gmail.com>\r\n'
reply: b'250 OK\r\n'
reply: retcode (250); Msg: b'OK'
send: 'data\r\n'
reply: b'354 End data with <CR><LF>.<CR><LF>\r\n'
reply: retcode (354); Msg: b'End data with <CR><LF>.<CR><LF>'
data: (354, b'End data with <CR><LF>.<CR><LF>')
send: b'\r\nSubject: Hi there\r\nThis message is sent from Python.\r\n.\r\n'
reply: b'250 OK\r\n'
reply: retcode (250); Msg: b'OK'
data: (250, b'OK')
send: 'QUIT\r\n'
reply: b'221 Bye\r\n'
reply: retcode (221); Msg: b'Bye'


Compara la salida de la consola con el ejemplo de una comunicación SMTP [según la wikipedia](https://es.wikipedia.org/wiki/Protocolo_para_transferencia_simple_de_correo): (seguro que el tuyo tiene algún `retcode` de más, pero deberías encontrar bastantes similitudes)

```
S: 220 Servidor SMTP
C: HELO miequipo.midominio.com
S: 250 Hello, please to meet you
C: MAIL FROM: <yo@midominio.com>
S: 250 Ok
C: RCPT TO: <destinatario@sudominio.com>
S: 250 Ok
C: DATA
S: 354 End data with <CR><LF>.<CR><LF>
C: Subject: Campo de asunto
C: From: yo@midominio.com
C: To: destinatario@sudominio.com
C:
C: Hola,
C: Esto es una prueba.
C: Hasta luego.
C:
C: .
C: <CR><LF>.<CR><LF>
S: 250 Ok: queued as 12345
C: quit
S: 221 Bye
```

## Extendiendo el servidor smtp

Como el servidor smtp por defecto no nos ofrece mucha información por defecto de la información que recibe, vamos a extender la funcionalidad de este servidor para que muestre por consola más datos.

Usaremos la herencia para crearnos una clase que herede del módulo `smtpd.SMTPServer` y reescribiremos su método `process_message` para imprimir toda la información que le llega (tienes toda la info sobre ese módulo y esa función [aquí](https://docs.python.org/3/library/smtpd.html?highlight=smtpd#smtpd.SMTPServer.process_message))

1. Para ello, crea un fichero llamado `email-server.py` e incluye el siguiente código en él:

```python
import smtpd
import asyncore

class CustomSMTPServer(smtpd.SMTPServer):

    def process_message(self, peer, mailfrom, rcpttos, \
                        data, mail_options=None, rcpt_options=None):
        print('Receiving message from:', peer)
        print('Message addressed from:', mailfrom)
        print('Message addressed to  :', rcpttos)
        print('Message length        :', len(data))


# Arranca el servidor en un proceso asíncrono
server = CustomSMTPServer(('127.0.0.1', 1025), None)
asyncore.loop()
```

2. Arranca el servidor usando este fichero con el siguiente comando:
```
python ejercicios/email-server.py
```

3. Reenvía el mensaje anterior al servidor:

In [6]:
with smtplib.SMTP(smtp_server, port) as server:
    server.sendmail(sender_email, receiver_email, message)

Ahora verás más información en la consola, algo como esto:
    
```
Receiving message from: ('127.0.0.1', 50488)                                                                         Message addressed from: my@gmail.com                                                                                 Message addressed to  : ['your@gmail.com']                                                                           Message length        : 52 
```